In [1]:
import os
import ipywidgets as widgets
from IPython.display import display, clear_output, Javascript 


original_dir = specified_dir = "../All_tests_launcher/"

class HomeScreen:
    def __init__(self):
        self.tracking_taken_tests = {"Ans_Test": 0, "Math_Test": 0, "Memory_Test": 0, "Spatial_Test": 0}
        self.output = widgets.Output()

    def setup_test_interaction(self, url):
        def on_button_clicked(b):
            with self.output:
                clear_output()  # Clear the output of the current cell
                display(Javascript(f'window.open("{url}", "_self");'))
        return on_button_clicked

    def home_screen_widgets(self):
        home_text = widgets.HTML(
            value="<div style='text-align: center; font-size: 16px;'>"
                  "<h2>Welcome to the UCL BIOS0030 cognitive tasks main menu</h2>"
                  "<p>You will have a chance to complete four different cognitive tasks, each testing different abilities.<br>"
                  "You may wish to allocate 10-15 minutes to complete all of them. After completing each task, the widgets will be marked,"
                  "to help you keep track of your progress.<br>Most importantly, <u>take them with a grain of salt</u>. Just like humans, cognitive tests are not perfect.<br>"
                  "<b>Have fun!</b></div>",
            layout=widgets.Layout(justify_content='center')
        )
        display(home_text)

        base_url = "https://uclbios-c7accae8eb34.herokuapp.com/"
        test_urls = {
            "ANS_Test": f"{base_url}/ANS_Test",
            "Maths_Test": f"{base_url}/Maths_Test",
            "Memory_Test": f"{base_url}/Memory_Test",
            "Spatial_Test": f"{base_url}/Spatial_Test",
        }

        choices_dictionary = {
            "ANS_Test": {
                "photo": widgets.Image(value=open("ans.png", "rb").read(), format='png', width=150, height=150),
                "button": widgets.Button(description="ANS Test")
            },
            "Maths_Test": {
                "photo": widgets.Image(value=open("math_photo.png", "rb").read(), format='png', width=150, height=150),
                "button": widgets.Button(description="Maths Test")
            },
            "Memory_Test": {
                "photo": widgets.Image(value=open("memory.png", "rb").read(), format='png', width=150, height=150),
                "button": widgets.Button(description="Memory Test")
            },
            "Spatial_Test": {
                "photo": widgets.Image(value=open("puzzle.png", "rb").read(), format='png', width=150, height=150),
                "button": widgets.Button(description="Spatial Test")
            }
        }

        for test, widget in choices_dictionary.items():
            url = test_urls[test]
            widget["button"].on_click(self.setup_test_interaction(url))

        grid = widgets.GridspecLayout(2, 4)
        for i, (test, test_widgets) in enumerate(choices_dictionary.items()):
            grid[0, i] = test_widgets["photo"]
            grid[1, i] = test_widgets["button"]

        vboxed_grid = widgets.VBox([grid],
                                   layout=widgets.Layout(align_items='center', margin='100px 0 0 0'))

        display(vboxed_grid, self.output)

# Create an instance of HomeScreen and display the main menu
os.chdir(specified_dir)
home = HomeScreen()
home.home_screen_widgets()


HTML(value="<div style='text-align: center; font-size: 16px;'><h2>Welcome to the UCL BIOS0030 cognitive tasks …

Output()